# 1. Library

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
from pathlib import Path
import random
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
from keras.preprocessing import image
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras import models
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

from sklearn.metrics import classification_report, confusion_matrix

# 2. Mount the Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Read the dataframe

In [ ]:
cultivar_df_test = pd.read_csv('/content/drive/My Drive/MasterProject/cultivarname_test.xlsx',dtype=str)
dahlia_df_test = pd.read_csv('/content/drive/My Drive/MasterProject/dahlianame_test.xlsx',dtype=str)
helleborus_df_test = pd.read_csv('/content/drive/My Drive/MasterProject/helleborusname_test.xlsx',dtype=str)

In [ ]:
cultivar_df_test

,Genus,Class_name,File_name,File_path
0,Dahlia,Dahlia _Arabian Night_,Dahlia _Arabian Night_ 0004.jpg,/content/drive/My Drive/MasterProject/split_da...
1,Dahlia,Dahlia _Arabian Night_,aug_0_9110.jpg,/content/drive/My Drive/MasterProject/split_da...
2,Dahlia,Dahlia _Alauna Clair-obscur_,Dahlia _Alauna Clair-obscur_ 0005.jpg,/content/drive/My Drive/MasterProject/split_da...
3,Dahlia,Dahlia _Akita_,Dahlia _Akita_ 0004.jpg,/content/drive/My Drive/MasterProject/split_da...
4,Dahlia,Dahlia _Akita_,aug_0_7249.jpg,/content/drive/My Drive/MasterProject/split_da...
...,...,...,...,...
123,Helleborus,Helleborus _Molly_s White_,aug_0_564.jpg,/content/drive/My Drive/MasterProject/split_da...
124,Helleborus,Helleborus _Yellow Lady_,Helleborus _Yellow Lady_ 0004.jpg,/content/drive/My Drive/MasterProject/split_da...
125,Helleborus,Helleborus _Yellow Lady_,aug_0_6113.jpg,/content/drive/My Drive/MasterProject/split_da...
126,Helleborus,Helleborus _Walhelivor (Ivory Prince)_,Helleborus _Walhelivor (Ivory Prince)_ 0004.jpg,/content/drive/My Drive/MasterProject/split_da...


# Labels

In [ ]:
labels_genus = cultivar_df_test['Genus'].unique()
labels_cultivar = cultivar_df_test['Class_name'].unique()
labels_dahlia = dahlia_df_test['Class_name'].unique()
labels_helleborus = helleborus_df_test['Class_name'].unique()
print(len(labels_genus),len(labels_cultivar),len(labels_dahlia),len(labels_helleborus))

2 66 59 7


In [ ]:
image_size = 224
batch_size = 1

# Prediction on test dataset

Load the image

In [ ]:
image_path = "/content/drive/My Drive/MasterProject/test/Helleborus/Helleborus _Penny_s Pink_/hellebore-pennys-pink.jpg"
#image_path = "/content/drive/My Drive/MasterProject/test/Dahlia/Dahlia _Alauna Clair-obscur_/Dahlia 'Alauna Clair-obscur'.jpg"
#img_path = "/content/drive/My Drive/MasterProject/test/Dahlia/Dahlia _American Dawn_/dahlia-american-dawn-2.jpg"

In [ ]:
test_images = list(cultivar_df_test["File_path"])
test_images = random.sample(test_images, 10)

In [ ]:
test_images = ["/content/drive/My Drive/MasterProject/test/Helleborus/Helleborus _Penny_s Pink_/hellebore-pennys-pink.jpg"]

Create the function for coding the image

In [ ]:
def load_image(img_path, show=False):
  global name
  name = img_path.split("/")[-2]
  #print("The true name is ",name)

  img = image.load_img(img_path, target_size=(image_size, image_size))
  img_tensor = image.img_to_array(img)                    # (height, width, channels)
  img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
  img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

  if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

  return img_tensor

load the table for saving the result

In [ ]:
filepath_result_test = Path('/content/drive/My Drive/MasterProject/result_test.xlsx')
df_result_test = pd.read_csv(filepath_result_test)

In [ ]:
genus_algorithms = ['/content/drive/My Drive/MasterProject/model/DenseNet201 model for genus.h5',
                      '/content/drive/My Drive/MasterProject/model/Xception model for genus.h5',
                      '/content/drive/My Drive/MasterProject/model/InceptionResNetV2 model for genus.h5',
                      '/content/drive/My Drive/MasterProject/model/EfficientNetB7 model for genus.h5']
cultivar_algorithms = ['/content/drive/My Drive/MasterProject/model/DenseNet201 model for cultivar.h5',
                      '/content/drive/My Drive/MasterProject/model/Xception model for cultivar.h5',
                      '/content/drive/My Drive/MasterProject/model/InceptionResNetV2 model for cultivar.h5',
                      '/content/drive/My Drive/MasterProject/model/EfficientNetB7 model for cultivar.h5']
dahlia_algorithms = ['/content/drive/My Drive/MasterProject/model/DenseNet201 model for Dahlia.h5',
                      '/content/drive/My Drive/MasterProject/model/Xception model for Dahlia.h5',
                      '/content/drive/My Drive/MasterProject/model/InceptionResNetV2 model for Dahlia.h5',
                      '/content/drive/My Drive/MasterProject/model/EfficientNetB7 model for Dahlia.h5']
helleborus_algorithms = ['/content/drive/My Drive/MasterProject/model/DenseNet201 model for Helleborus.h5',
                         '/content/drive/My Drive/MasterProject/model/Xception model for Helleborus.h5',
                         '/content/drive/My Drive/MasterProject/model/InceptionResNetV2 model for Helleborus.h5',
                         '/content/drive/My Drive/MasterProject/model/EfficientNetB7 model for Helleborus.h5']

In [ ]:
def save_result():
  filepath_result_end = Path('/content/drive/My Drive/MasterProject/result_end.xlsx')
  df_result_end.to_csv(filepath_result_end,index=False)

In [ ]:
df_result_end = pd.DataFrame(columns=['True Name',
                                  'Name of a Plant',
                                   '%',])
filepath_result_end = Path('/content/drive/My Drive/MasterProject/result_end.xlsx')
df_result_end.to_csv(filepath_result_end,index=False)

In [ ]:
def fff(labels,algorithms,image_path):
  global df
  new_image = load_image(image_path)
  #keras.backend.clear_session()  
  df_result_test = pd.read_csv('/content/drive/My Drive/MasterProject/result_test.xlsx') 
  for path_model in algorithms:
    model = load_model(path_model)
    pred = model.predict(new_image)
    predictions = np.argmax(pred, axis=1)

    result = {labels[i]: pred[0][i] for i in range(len(labels))}
    sort_result = sorted(result.items(), key=lambda x: x[1], reverse=True)
    
    for i in sort_result[:5]:
      #print(i[0], round((i[1]*100),2), "%")
      
      df_result_test.loc[len(df_result_test.index)] = [i[0], round((i[1]*100),2)]
    #keras.backend.clear_session()  
  
  df = df_result_test.groupby(['Name of a Plant']).sum().sort_values("%", ascending=False)
  df.reset_index(inplace=True) 
  df.to_csv('/content/drive/My Drive/MasterProject/df.xlsx')
   
  #print("df is: ", df)
  
  df_result_end.loc[len(df_result_end.index)] = [name,
                                                 df.iat[0,0],
                                                 df.iat[0,1]]
  df_result_end.to_csv('/content/drive/My Drive/MasterProject/result_end.xlsx')
  
  #print(df.iat[0,0])
  
  return df

In [ ]:
for path in test_images:
  fff(labels_genus,genus_algorithms,path)
    
  if df.iat[0,0] =='Dahlia':
    #if df.iloc[0][0] == "Dahlia":
    fff(labels_dahlia,dahlia_algorithms,path)
  elif df.iat[0,0] =='Helleborus':
    fff(labels_helleborus,helleborus_algorithms,path)
  else:
    print("problem")
print(df_result_end)  

df is:    Name of a Plant       %
0          Dahlia  389.28
1      Helleborus   10.72
Dahlia
df is:                 Name of a Plant      %
0             Dahlia _Maldini_  25.58
1          Dahlia _Daisy Duke_  20.70
2   Dahlia _Gallery Rembrandt_  18.18
3               Dahlia _Pablo_  10.39
4    Dahlia _Crème de Cognac_  10.31
5           Dahlia _Chat Noir_   9.25
6               Dahlia _Akita_   8.46
7            Dahlia _Rip City_   7.42
8        Dahlia _Cornel Brons_   7.20
9      Dahlia _Fashion Monger_   5.93
10   Dahlia _Lindsay Michelle_   5.89
11    Dahlia _Rebecca_s World_   5.72
12   Dahlia _Kelsey Annie Joy_   5.18
13            Dahlia _Tam Tam_   2.53
14     Dahlia _Bishop of York_   2.35
15    Dahlia _Bishop of Dover_   2.17
16      Dahlia _Arabian Night_   2.10
Dahlia _Maldini_
df is:    Name of a Plant       %
0          Dahlia  389.31
1      Helleborus   10.69
Dahlia
df is:                    Name of a Plant       %
0       Dahlia _Crème de Cassis_  106.72
1            

In [ ]:
for path in test_images:
  fff(labels_cultivar,cultivar_algorithms,path)
print(df_result_end)

df is:                 Name of a Plant      %
0    Dahlia _Crème de Cognac_  46.04
1             Dahlia _Maldini_  43.54
2      Dahlia _Fashion Monger_  29.13
3          Dahlia _Daisy Duke_  22.88
4   Dahlia _Gallery Rembrandt_  17.59
5               Dahlia _Akita_   8.29
6         Dahlia _White Aster_   6.85
7    Dahlia _Lindsay Michelle_   6.36
8    Dahlia _Gallery Art Deco_   5.97
9           Dahlia _Chat Noir_   5.51
10    Dahlia _Rebecca_s World_   5.16
11            Dahlia _Tam Tam_   2.47
12              Dahlia _Pablo_   1.84
13            Dahlia _Seattle_   1.81
Dahlia _Crème de Cognac_
df is:                    Name of a Plant       %
0       Dahlia _Crème de Cassis_  150.77
1            Dahlia _White Aster_   59.57
2            Dahlia _Brown Sugar_   32.57
3   Dahlia _Tsuki-yori-no-shisha_   22.51
4           Dahlia _Linda_s Baby_   14.88
5             Dahlia _Daisy Duke_    5.80
6                Dahlia _Tam Tam_    2.47
7       Dahlia _Gallery Art Deco_    1.94
8         

In [ ]:
#df_result_end.loc[20:30,["Name of a Plant","%"]] = df_result_end[0:11,['Predicting name for cultivar',"%"]]
df_result_end['Predicting name for cultivar'] = df_result_end.loc[20:30,["Name of a Plant"]]
df_result_end['% '] = df_result_end.loc[20:30,["%"]]
df_result_end

,True Name,Name of a Plant,%,Predicting name for cultivar,%
0,Dahlia _Melody Harmony_,Dahlia,389.28,NaN,NaN
1,Dahlia _Melody Harmony_,Dahlia _Maldini_,25.58,NaN,NaN
2,Dahlia _Cornel Brons_,Dahlia,389.31,NaN,NaN
3,Dahlia _Cornel Brons_,Dahlia _Crème de Cassis_,106.72,NaN,NaN
4,Dahlia _Bishop of Oxford_,Dahlia,389.20,NaN,NaN
5,Dahlia _Bishop of Oxford_,Dahlia _Bishop of Llandaff_,114.30,NaN,NaN
6,Helleborus _Anna_s Red_,Helleborus,310.42,NaN,NaN
7,Helleborus _Anna_s Red_,Helleborus _Anna_s Red_,224.83,NaN,NaN
8,Dahlia _Bishop of York_,Dahlia,389.27,NaN,NaN
9,Dahlia _Bishop of York_,Dahlia _Black Jack_,120.27,NaN,NaN
